In [ ]:
# dependencies import
from common_dependencies import *
from itertools import chain
import logging
logger = logging.getLogger('main.make_pred.ipynb')

In [ ]:
# paths for files with original data
PATH_TO_DATA = {
    'run_1': 
        ('data/original_data/run_1/run1_WM32_data.csv',
        'data/original_data/run_1/run1_WM32_defects.csv',
        'data/original_data/run_1/run1_WM32_pipe.csv'),
    'run_2':
        ('data/original_data/run_2/run2_WM32_data.csv',
        'data/original_data/run_2/run2_WM32_defects.csv',
        'data/original_data/run_2/run2_WM32_pipe.csv')
}


PATH_TO_MODEL = 'networks/CNN'
RUNS = [2, 1]
MODEL_VER = '14'
MODEL_NUM = '01'
XSHIFT = 200

for name in os.listdir(PATH_TO_MODEL):
    res = re.match(F'(id=v{MODEL_VER}n{MODEL_NUM}).*', name)
    if not res is None:
        PATH_TO_MODEL += '/' + res[0]
        break

# детерминация случайных величин, отвечающих за выбор первоначальных весов и биасов
tf.compat.v1.set_random_seed(290)
tf.random.set_seed(290)

In [ ]:
# Загрузка модели
load_model = keras.models.load_model(PATH_TO_MODEL)
CROP_SIZE = load_model.inputs[0].shape[1]
CROP_STEPS = [CROP_SIZE, CROP_SIZE//2, CROP_SIZE//4, CROP_SIZE//8]

In [ ]:
def make_pred_fast(model, 
                   path_to_data_tuple: tuple(),
                   x_shift: int,
                   crop_size: int, 
                   crop_step: int):

    x_df, y_df = dw.get_x_and_y_data(*path_to_data_tuple)

    x_df = dw.roll_df(x_df, x_shift, 1)
    y_df = dw.roll_df(y_df, x_shift, 1)

    or_rows = x_df.shape[0]
    or_cols = x_df.shape[1]
    ex_x_df = dw.extend_df_for_prediction(x_df, crop_size)
    ex_x_df = dw.extend_df_for_crops_dividing(ex_x_df, crop_size, crop_step)
    ex_y_df = dw.extend_df_for_prediction(y_df, crop_size)
    ex_y_df = dw.extend_df_for_crops_dividing(ex_y_df, crop_size, crop_step)
    ex_rows = ex_x_df.shape[0]
    ex_cols = ex_x_df.shape[1]
    
    x_arr = dw.df_to_numpy(ex_x_df)
    y_arr = ex_y_df.to_numpy()
    
    #x_arr_time = dw.standardize_data(x_arr[:,:,:32])
    #x_arr_amp = dw.standardize_data(x_arr[:,:,32:])

    x_arr = np.concatenate([dw.normalize_data(x_arr[:,:,:32]), dw.normalize_data(x_arr[:,:,32:])],axis=2)
    y_arr = dw.normalize_data(y_arr)
    
    #x_arr_time_crops_gen = dw.get_crop_generator(x_arr_time, crop_size, crop_step)
    #x_arr_amp_crops_gen = dw.get_crop_generator(x_arr_amp, crop_size, crop_step)
    
    x_crops_gen = dw.get_crop_generator(x_arr, crop_size, crop_step)
    y_binary_gen = (1 if np.sum(crop > 0) else 0 for crop in dw.get_crop_generator(y_arr, crop_size, crop_step))
    #y_data_depth_gen = (np.max(crop) for crop in dw.get_crop_generator(y_arr, crop_size, crop_step))

    
    
    #x_time = np.stack([crop for crop in x_arr_time_crops_gen])
    #x_amp = np.stack([crop for crop in x_arr_amp_crops_gen])
    x_data = np.array(list(x_crops_gen))
    y_binary = np.array(list(y_binary_gen))
    #y_depth = np.array([depth for depth in y_data_depth_gen])

    # 1 output
    res = model.predict(x_data)[:,0]
    res_it = iter(res)

    # 2 outputs
    '''res = np.array(model.predict([x_data_time, x_data_amp]))
    res = np.squeeze(res, axis=2)
    bool_res = res[0,:]
    depth_res = res[1,:] 
    bool_res_it = iter(bool_res) 
    depth_res_it = iter(depth_res) 
    res_it = bool_res_it'''
    
    def fill_by_arrays(df_cell_value):
        return list()
    def add_number_to_arrays(df_cell_value, number):
        df_cell_value.append(number)
        return df_cell_value
    extend_size_result_df = pd.DataFrame(data=0, 
                             columns=ex_x_df.columns, 
                             index=ex_x_df.index)
    extend_size_result_df = extend_size_result_df.map(fill_by_arrays)

    for i in range(0, ex_rows - crop_size + 1, crop_step):
        for j in range(0,  ex_cols - crop_size + 1, crop_step):
            temp_add = next(res_it)
            extend_size_result_df.iloc[i:i+crop_size, j:j+crop_size] = \
                extend_size_result_df.iloc[i:i+crop_size, j:j+crop_size].map(lambda x: add_number_to_arrays(x, temp_add))

    orig_size_result_df = extend_size_result_df.iloc[crop_size-1:, crop_size-1:].iloc[:or_rows, :or_cols]
    
    return orig_size_result_df, extend_size_result_df, y_df, ex_y_df 

In [ ]:
for run in RUNS:
    for crop_step in CROP_STEPS:
        logger.debug(f'run: {run}, crop step: {crop_step}')
        (orig_size_result_df, 
        extend_size_result_df,
        orig_size_reference_df, 
        extend_size_reference_df) = make_pred_fast(load_model, PATH_TO_DATA[f'run_{run}'], XSHIFT, CROP_SIZE, crop_step)

        def calc_quartile(df_cell):
            q1 = [item for item in df_cell if item < 0.25]
            q2 = [item for item in df_cell if item >= 0.25 and item < 0.5]
            q3 = [item for item in df_cell if item >= 0.5 and item < 0.75]
            q4 = [item for item in df_cell if item >= 0.75]
            qrt = [len(q1), len(q2), len(q3), len(q4)]
            ind = qrt.index(max(qrt))
            if ind == 0:
                return np.mean(q1)
            elif ind == 1:
                return np.mean(q2)
            elif ind == 2:
                return np.mean(q3)
            else:
                return np.mean(q4)

        def calc_half(df_cell):
            h1 = [item for item in df_cell if item < 0.5]
            h2 = [item for item in df_cell if item >= 0.5]
            qrt = [len(h1), len(h2)]
            ind = qrt.index(max(qrt))
            
            if ind == 0:
                return np.mean(h1)
            else:
                return np.mean(h2)
        
        dw.draw_defects_map(orig_size_reference_df, title = f'REFERENCE run: {run}, crop step: {crop_step}, crop size: {CROP_SIZE}')
        dw.draw_defects_map(orig_size_result_df.map(calc_quartile), title = f'PREDICTION QUART run: {run}, crop step: {crop_step}, crop size: {CROP_SIZE}')
        dw.draw_defects_map(orig_size_result_df.map(calc_half), title = f'PREDICTION HALF run: {run}, crop step: {crop_step}, crop size: {CROP_SIZE}')
        dw.draw_defects_map(orig_size_result_df.map(lambda x: np.max(x)), title = f'PREDICTION MAX run: {run}, crop step: {crop_step}, crop size: {CROP_SIZE}')
        dw.draw_defects_map(orig_size_result_df.map(lambda x: np.min(x)), title = f'PREDICTION MIN run: {run}, crop step: {crop_step}, crop size: {CROP_SIZE}')

                
        #display(orig_size_result_df)
        '''path_to_run = f'data/drawing_data/model_id=v{MODEL_VER}n{MODEL_NUM}/run_{run}/' 
        res_file_name = f'model_id=v{MODEL_VER}n{MODEL_NUM}_crop(size={CROP_SIZE},step={crop_step})_shift(x={XSHIFT},y=0).xlsx'
        
        if not os.path.exists(path_to_run):
            os.makedirs(path_to_run)
        
        with pd.ExcelWriter(os.path.join(path_to_run, res_file_name)) as writer:  
            orig_size_result_df.to_excel(writer, sheet_name='orig_size_result')
            extend_size_result_df.to_excel(writer, sheet_name='extend_size_result')
            
            orig_size_reference_df.to_excel(writer, sheet_name='orig_size_reference')
            extend_size_reference_df.to_excel(writer, sheet_name='extend_size_reference')'''